In [ ]:
import os
import json
import shutil
import yaml
from PIL import Image

yaml_path = r"data.yaml"
with open(yaml_path, "r") as f:
    yolo_yaml = yaml.safe_load(f)
classes = yolo_yaml["names"] 

yolo_base = r"C:\Users\rouat\Desktop\MMcows\Datasets\yolo"
yolo_image_train = os.path.join(yolo_base, "images", "train")
yolo_label_train = os.path.join(yolo_base, "labels", "train")
yolo_image_val = os.path.join(yolo_base, "images", "val")
yolo_label_val = os.path.join(yolo_base, "labels", "val")
yolo_image_test = os.path.join(yolo_base, "images", "test")
yolo_label_test = os.path.join(yolo_base, "labels", "test")

effdet_base = r"C:\Users\rouat\Desktop\MMcows\Datasets\efficientdet"
train_img_dst = os.path.join(effdet_base, "train")
val_img_dst = os.path.join(effdet_base, "val")
test_img_dst = os.path.join(effdet_base, "test")
ann_dir = os.path.join(effdet_base, "annotations")
os.makedirs(train_img_dst, exist_ok=True)
os.makedirs(val_img_dst, exist_ok=True)
os.makedirs(test_img_dst, exist_ok=True)
os.makedirs(ann_dir, exist_ok=True)

def count_images_labels(image_dir, label_dir):
    images = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
    ann_count = 0
    for img_name in images:
        label_file = os.path.join(label_dir, os.path.splitext(img_name)[0] + ".txt")
        if os.path.exists(label_file):
            with open(label_file, "r") as f:
                ann_count += sum(1 for _ in f)
    return len(images), ann_count

def convert_split(image_dir, label_dir, output_img_dir, output_json_path, starting_image_id=1, starting_ann_id=1):
    coco = {
        "images": [],
        "annotations": [],
        "categories": []
    }

    for i, class_name in enumerate(classes):
        coco["categories"].append({
            "id": i + 1,
            "name": class_name,
            "supercategory": "none"
        })

    ann_id = starting_ann_id
    img_id = starting_image_id

    for filename in os.listdir(image_dir):
        if not filename.lower().endswith((".jpg", ".png", ".jpeg")):
            continue

        img_path = os.path.join(image_dir, filename)
        try:
            img = Image.open(img_path)
            width, height = img.size
        except Exception as e:
            print(f" Erreur ouverture image {filename}: {e}")
            continue

        shutil.copy(img_path, os.path.join(output_img_dir, filename))

        coco["images"].append({
            "id": img_id,
            "file_name": filename,
            "width": width,
            "height": height
        })

        label_file = os.path.join(label_dir, os.path.splitext(filename)[0] + ".txt")
        if os.path.exists(label_file):
            with open(label_file, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        continue
                    class_id, x_center, y_center, w, h = map(float, parts)

                    x = (x_center - w / 2) * width
                    y = (y_center - h / 2) * height
                    w *= width
                    h *= height

                    coco["annotations"].append({
                        "id": ann_id,
                        "image_id": img_id,
                        "category_id": int(class_id) + 1,
                        "bbox": [x, y, w, h],
                        "area": w * h,
                        "iscrowd": 0
                    })
                    ann_id += 1
        img_id += 1

    with open(output_json_path, "w") as f:
        json.dump(coco, f, indent=4)
    print(f"✅ COCO JSON généré : {output_json_path}")

train_img_count, train_ann_count = count_images_labels(yolo_image_train, yolo_label_train)
val_img_count, val_ann_count = count_images_labels(yolo_image_val, yolo_label_val)

convert_split(
    yolo_image_train,
    yolo_label_train,
    train_img_dst,
    os.path.join(ann_dir, "instances_train.json"),
    starting_image_id=1,
    starting_ann_id=1
)

convert_split(
    yolo_image_val,
    yolo_label_val,
    val_img_dst,
    os.path.join(ann_dir, "instances_val.json"),
    starting_image_id=train_img_count + 1,
    starting_ann_id=train_ann_count + 1
)

convert_split(
    yolo_image_test,
    yolo_label_test,
    test_img_dst,
    os.path.join(ann_dir, "instances_test.json"),
    starting_image_id=train_img_count + val_img_count + 1,
    starting_ann_id=train_ann_count + val_ann_count + 1
)


✅ COCO JSON généré : C:\Users\rouat\Desktop\MMcows\Datasets\efficientdet\annotations\instances_train.json
✅ COCO JSON généré : C:\Users\rouat\Desktop\MMcows\Datasets\efficientdet\annotations\instances_val.json
✅ COCO JSON généré : C:\Users\rouat\Desktop\MMcows\Datasets\efficientdet\annotations\instances_test.json
